In [1]:
! pip install beautifulsoup4 lxml

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import json
from bs4 import BeautifulSoup # HTML에서 날짜 정보를 추출하기 위해 사용
from datetime import datetime # 문자열 날짜를 YYYY-MM-DD 형식으로 변환
from collections import defaultdict # 날짜별 포스팅 수를 자동으로 카운팅하는 딕셔너리


# 백업 폴더 설정 및 반복
base_path = 'catherine_backup(250613)' # 백업 폴더 경로 설정
date_count = defaultdict(int) # 날짜별 포스팅 수를 저장할 딕셔너리 (기본값 0)

# 백업 폴더 안의 모든 폴더를 순회
for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)

    # 폴더 이름이 숫자이고 실제 폴더일 경우에만 처리
    if os.path.isdir(folder_path) and folder_name.isdigit():

        # 해당 폴더 안의 파일을 모두 순회
        for file_name in os.listdir(folder_path):

             # .html 파일만 처리
            if file_name.endswith('.html'):
                file_path = os.path.join(folder_path, file_name)

                 # HTML 파일 열기
                with open(file_path, 'r', encoding='utf-8') as f:
                    soup = BeautifulSoup(f, 'html.parser')
                    date_tag = soup.find('p', class_='date')  # <p class="date"> 태그 찾기
                     # 날짜 태그가 존재하면 처리
                    if date_tag:
                        raw_date = date_tag.text.strip()  # 날짜 문자열 추출
                        try:
                            # # "YYYY-MM-DD HH:MM:SS" 형식을 "YYYY-MM-DD"로 변환
                            date = datetime.strptime(raw_date, "%Y-%m-%d %H:%M:%S").date().isoformat()
                            date_count[date] += 1 # 해당 날짜의 포스팅 수 +1 증가
                        except ValueError:
                            continue  # 날짜 형식이 이상할 경우 무시하고 넘어

# 결과를 날짜순으로 정렬해서 JSON 파일로 저장
with open('backup_date.json', 'w', encoding='utf-8') as f:
    json.dump(dict(sorted(date_count.items())), f, ensure_ascii=False, indent=2)
